In [1]:
import scapy.all as scapy
import pandas as pd
from scapy.layers.inet import IP, TCP, UDP, ICMP
from scapy.layers.l2 import ARP, Ether, Dot3
from scapy.layers.dns import DNS
from scapy.layers.http import HTTP
from scapy.packet import Raw
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Define the features to be extracted from the pcap file
def extract_features(packet):
    features = []
    # Extract the packet length
    features.append(len(packet))
    # Extract the source IP and destination IP, if present
    if IP in packet:
        features.append(packet[IP].src)
        features.append(packet[IP].dst)
    else:
        features.extend([0, 0])
    # Extract the source port and destination port, if present
    if TCP in packet:
        features.append(packet[TCP].sport)
        features.append(packet[TCP].dport)
    elif UDP in packet:
        features.append(packet[UDP].sport)
        features.append(packet[UDP].dport)
    else:
        features.extend([0, 0])
    # Extract the protocol, if present
    if IP in packet:
        features.append(packet[IP].proto)
    else:
        features.append(0)
    # Extract the payload content, if present
    if TCP in packet:
        payload = str(packet[TCP].payload)
        features.append(payload)
    else:
        features.append(0)
    # Extract the connection duration
    global previous_packet_time
    connection_duration = packet.time - previous_packet_time
    features.append(connection_duration)
    previous_packet_time = packet.time
    return features

# Read the pcap file and extract features from each packet
packets = scapy.rdpcap('final.pcap')
previous_packet_time = packets[0].time
X = []
for packet in packets:
    features = extract_features(packet)
    X.append(features)

# Convert the extracted features to a pandas dataframe
columns = ['Packet Length', 'Source IP', 'Destination IP', 'Source Port', 'Destination Port', 'Protocol', 'Payload', 'Connection Duration']
df = pd.DataFrame(X, columns=columns)

print (df)


# Preprocess the data by converting categorical variables to numerical variables and scaling the data
categorical_features = ['Source IP', 'Destination IP', 'Protocol']
df = pd.get_dummies(df, columns=categorical_features)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df.drop(['Payload'], axis=1))

# Train a Random Forest Classifier model on the preprocessed data
y = []

# Assuming the following types of traffic are malicious: DNS, DoS, Land, DDoS, Smurf, ARP Poisoning, ARP Spoofing, Nmap, TCP Scan, UDP Scan, Full Scan, Half Scan, Xmas Scan
for packet in packets:

    # Scan
    if packet.haslayer(TCP) and packet[TCP].flags.F == 1 and packet[TCP].flags.S == 0 and packet[TCP].flags.A == 0:
        y.append(1)
    
    elif packet.haslayer(IP) and packet[IP].src == packet[IP].dst:
        y.append(1)

    # ICMP Flood
    elif packet.haslayer(ICMP) and packet[ICMP].type == 8 and packet[ICMP].code == 0:
        y.append(1)

    # detected Ping of Death attack
    elif packet.haslayer(ICMP) and len(packet) > 65535:
        y.append(1)
     
    # detected TCP SYN Flooding attack
    elif packet.haslayer(TCP) and packet[TCP].flags.S == 1 and packet[TCP].flags.A == 0:
        y.append(1)

    # detected UDP Flooding attack
    elif packet.haslayer(UDP) and len(packet) > 65535:
        y.append(1)

    # detected TCP Flooding attack
    elif packet.haslayer(TCP) and len(packet) > 65535:
        y.append(1)
    
    #detected DNS amplification attack
    elif packet.haslayer(DNS) and packet[DNS].qr == 0 and packet[DNS].opcode == 0 and packet[DNS].ancount == 0:
        y.append(1)
     
    # detected Slowloris attack
    elif packet.haslayer(TCP) and packet[TCP].flags.S == 1 and packet[TCP].flags.A == 0 and packet.time_delta > 10:
        y.append(1)

    # detected DDoS attack
    elif packet.haslayer(TCP) and packet[TCP].flags.S == 1 and packet[TCP].flags.A == 0 and packet.time_delta < 0.01:
        y.append(1)

    # SQL Injection
    elif packet.haslayer(TCP) and packet[TCP].dport == 80 and "SELECT" in packet[Raw].load:
        y.append(1)
     
    # Xmas Scan
    elif packet.haslayer(TCP) and packet[TCP].flags.F == 1 and packet[TCP].flags.S == 1 and packet[TCP].flags.A == 0:
        y.append(1)

    # Ping Flood
    elif packet.haslayer(ICMP) and packet[ICMP].type == 8:
        if len(set([pkt[IP].src for pkt in packets if pkt.haslayer(ICMP) and pkt[ICMP].type == 8])) > 5:
            y.append(1)
            
    # Smurf Attack
    elif packet.haslayer(IP) and packet[IP].dst == '255.255.255.255':
        if packet.haslayer(ICMP) and packet[ICMP].type == 8:
            y.append(1)

    #ARP Poisoning
    elif packet.haslayer(ARP) and packet[ARP].op == 2 and packet[ARP].hwsrc != packet[ARP].psrc:
        y.append(1)

    # TCP Scan
    elif packet.haslayer(TCP) and packet[TCP].flags == 0x02:
        y.append(1)

    # Full Scan
    elif packet.haslayer(TCP) and packet[TCP].flags == 0x1f:
        y.append(1)

    # Half Scan
    elif packet.haslayer(TCP) and packet[TCP].flags == 0x12:
        y.append(1)

    # Xmas Scan
    elif packet.haslayer(TCP) and packet[TCP].flags == 0x29:
        y.append(1)

    else:
        y.append(0)
        

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_scaled, y)

# Predict the class of each packet using the trained model
predictions = clf.predict(X_scaled)

# Print the results
# Print the results
for i in range(len(packets)):
    if i == 6971:
        if predictions[i] == 1:
            print(f"Packet {i} is malicious")
            print (packets[i].show())
        elif predictions[i] == 0:
            print(f"Packet {i} is not malicious")


      Packet Length       Source IP  Destination IP  Source Port  \
0                54    3.68.124.168  192.168.100.26          443   
1                54  192.168.100.26    3.68.124.168        53839   
2               110  192.168.100.26    3.108.30.176        53603   
3                54    3.108.30.176  192.168.100.26          443   
4               108  192.168.100.26    3.108.30.176        53603   
...             ...             ...             ...          ...   
6987            373      20.111.1.3  192.168.100.26          443   
6988             54  192.168.100.26      20.111.1.3        53978   
6989            212  192.168.100.26      20.111.1.3        53978   
6990            911  192.168.100.26      20.111.1.3        53978   
6991            103               0               0        60982   

      Destination Port  Protocol Payload Connection Duration  
0                53839         6                    0.000000  
1                  443         6                    0.000

In [2]:
# Importing required libraries
import joblib

# Saving the trained model as a file
joblib.dump(clf, 'model.pkl')

['model.pkl']